#### Crucial note (working of this code is dependant)
- The working of this notebook is conditioned on the `uncertain_IDM` model in `AutomotiveDrivingModels
which implements the sensor noise to corrupt headway measurement

#### Changelog
- 30 Sep 2019
    - Creation
    - Goal is to do MOBIL experimentation to demonstrate lane changes
    - Idea flow
        - Look at some videos to select a situation with enough spacing to make lane change
        - Select a vehicle id that is on a slow lane
        - Make it want to make a lange change by some selection of parameters
- 2 Oct
    - Defining in notebook copies of `MOBIL` and `Tim2DDriver` for debugging purposed
    - Above did not work because methods being redefined not working (or something like that)
    - Found that Maxime had a commit on June 18 fixing MOBIL.jl. Did a git pull 
    on `.julia/packages/dev/AutomotiveDrivingModels`
    - Seems to change lane
- 3 Oct
    - Influence of stochastictity on the `ProportionalLaneTracker`
    - trace the x and y positions of car
- 10 Oct
    - Function to get the lane id
- 16 Oct
    - Back to working in this notebook now that upstream dependencies in `AutomotiveDrivingModels.jl`
    have solidified
    - `uncertain_IDM.jl` is a new driver model that has been defined in `AutomotiveDrivingModels/src/behaviors`
    - Global things defined at top of notebook to step away from dictionary based particles
- 17 Oct
    - Uniform sampling to create initial particle matrix
- 18 Oct
    - Likelihood computation works
    - Renamed `compute_particle_likelihoods` to `weight_and_resample` as combined it with previous
    function called `update_p_one_step`
- 22 Oct
    - `multistep_update` works without error
    - rmse computation functions written to compare ground truth versus imitation trajectory
    - Script written for end to end: filtering on ground truth, imitation traj generation, rmse compute,
    and overlay video
- 28 Oct
    - Made particle progress plot function to plot along with error bars optionally
    - Script to generate multiple imitation trajs using mean of filtered particle matrix
- 29 Oct
    - Copied all functions into `utils.jl` to enable use of
    `debug_lane_change.jl` to explain variability in second lane change
    - Changed `gen_imitation_traj` other vehicle i.e id 2 and 3 to be driven by IDM (v_des = 15)
    to match the ground truth generation process (was earlier being done as vdes = 50.) to match
    the hallucination process
    - DEFAULT_PARAMS stores the default values of the parameters used to generate the ground truth trajectory
- 30 Oct
    - Noticed that ground truth vehicle 3 was being driven by default IDM (and not v_des = 15) so made
    the same thing true for generate imitation i.e. veh 3 driven by default instead of vdes 15 IDM
- 31 Oct
    - Changed `gen_imitation_traj` to take a timestep kwarg after discussion with Alex about Carla
    - Also changed it to take `duration` as a kwarg instead of `nsteps` and do the requisite nsteps calc itself
    - Script to convert scenes into data that can be written to csv to give to Alex
- 4 Nov
    - Cleaned up this notebook into sections
    - Copied this notebook to started a new notebook called `mobil_ngsim` to work with ngsim data
- 14 Nov
    - Pairwise progress of particle filtering update and making associated gif
        - Had to do some fixing to make Reel work with PGFPlots
    
#### Basic idea of what this notebook does
- Generate initial sample set of particles given some limits on the possible range of each of the parameters
- Use a trajectory of driving behavior as the ground truth and run filtering using sample set of particles
- Get a set of filtered particles. Take the mean of these particles and use it to generate an imitation trajectory
- Evaluate rmse of that imitation trajectory with respect to the ground truth demonstration trajectory

In [1]:
# using packages
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Reel
using Distributions
using Random
using PGFPlots
using LinearAlgebra # To make the MvNormal covariance work maybe
using JLD
using StatsBase # For weighting particles according to likelihoods and then resampling
using DelimitedFiles # For writing to csv
using Combinatorics # For pairwise parameter selection to make progress plot videos

┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/NGSIM/unWOy.ji for NGSIM [195157bc-cedd-5875-9ea7-7cfb75f30b3c]
└ @ Base loading.jl:1184
┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = lstirling_asym(::BigFloat) at misc.jl:56
└ @ StatsFuns ~/.julia/packages/StatsFuns/2QE7p/src/misc.jl:56
┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/AutoViz/w0rHu.ji for AutoViz [82aa6e0c-a491-5edf-8d4b-c16b98e4ea17]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/PGFPlots/XxINN.ji for PGFPlots [3b7a836e-365b-5785-a47d-02c71176b4aa]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/JLD/l8ijy.ji for JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base loading.jl:1184


In [2]:
# overlays: IDOverlay. my_overlay
"""
    IDOverlay
Display the ID on top of each entity in a scene.
# Fields
- `color::Colorant`
- `font_size::Int64`
"""
mutable struct IDOverlay <: SceneOverlay
    color::Colorant
    font_size::Int
end

function AutoViz.render!(rendermodel::RenderModel, overlay::IDOverlay, scene::Scene, 
                            env::E) where E
    font_size = overlay.font_size
    for veh in scene
        add_instruction!(rendermodel, render_text, ("$(veh.id)", veh.state.posG.x, 
                        veh.state.posG.y, font_size, overlay.color), incameraframe=true)
    end
    return rendermodel
end

"""
    my_overlay
Overlaying hallucinated trajectory on the ground truth
# Fields
- `color::Colorant`
- `scene::Scene`
"""
struct my_overlay <: SceneOverlay
    scene::Scene
    color # Needs to be of form colorant"Colorname"
end

function AutoViz.render!(rendermodel::RenderModel,overlay::my_overlay, 
        scene::Scene, roadway::Roadway)
    AutoViz.render!(rendermodel,overlay.scene,car_color = overlay.color)
    return rendermodel
end

In [3]:
# functions: get_frenet_s, get_lane_id, lane change probability
"""
    function get_frenet_s(scene;car_id=-1)

# Examples
```julia
true_next_pos = get_frenet_s(true_next_scene,car_id=1)
```
"""
function get_frenet_s(scene;car_id=-1)
    if car_id==-1 print("get_frenet_s says: Give valid car id") end
    veh = scene[findfirst(car_id,scene)]
    return veh.state.posF.s
end

"""
    function get_lane_id(scene,car_id)
# Examples
```julia
get_lane_id(scene,1)
```
"""
function get_lane_id(scene,car_id)
    veh = scene[findfirst(car_id,scene)]
    return veh.state.posF.roadind.tag.lane
end

"""
    function get_lane_change_prob(start_scene,particle;car_id=-1,num_samplings=10)
- Probability of lane changing start from `start_scene`
- hallucinating using `particle` for `car_id` using `num_samplings` hallucinations

# Examples
```julia
lp = get_lane_change_prob(scene,particle,car_id = 1)
```
"""
function get_lane_change_prob(start_scene,particle;car_id=-1,num_samplings=10)
    if car_id==-1 @show "get_lane_change_prob says: Please give valid car_id" end
    start_lane = get_lane_id(start_scene,car_id)
    changed_count = 0; unchanged_count = 0
    for i in 1:num_samplings
        hpos,hlane = hallucinate_a_step(start_scene,particle,car_id=car_id)
        if hlane == start_lane
            unchanged_count += 1
	else
	    changed_count += 1
	end
    end
    return (changed_count+1)/(num_samplings+2)
end

get_lane_change_prob

In [4]:
# function: generate roadway and place cars
"""
    function init_place_cars(lane_place_array;road_length = 1000.0)
- Place cars on a straight roadway of `road_length` according to elems in `lane_place_array`

# Examples
```julia
pos_vel_array_1 = [(200.,30.),(215.,0.),(220.,0.)]
pos_vel_array_2 = [(200.,0.),(215.,0.),(220.,20.)]
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2,pos_vel_array_3]
scene,roadway = init_place_cars(lane_place_array)
```
"""
function init_place_cars(lane_place_array;road_length = 1000.0)
    num_lanes = length(lane_place_array)
    roadway = gen_straight_roadway(num_lanes,road_length)
    scene = Scene()

    id = 1
    for i in 1:num_lanes
        for j in 1:length(lane_place_array[i])
            veh_state = VehicleState(Frenet(roadway[LaneTag(1,i)],
                    lane_place_array[i][j][1]),roadway,
                lane_place_array[i][j][2])
            veh = Vehicle(veh_state,VehicleDef(),id)
            push!(scene,veh)
            id+=1
        end
    end
    return scene,roadway
end

init_place_cars

In [5]:
# function: videos
"""
    function scenelist2video(scene_list;filename = "media/mobil/scene_to_video.mp4")
- Make video from a list of scenes

# Examples
```julia
scenelist2video(scene_list,filename="media/mobil/true_traj.mp4")
```
"""
function scenelist2video(scene_list;filename = "media/mobil/scene_to_video.mp4")
    frames = Frames(MIME("image/png"),fps = 10)
    
    # Loop over list of scenes and convert to video
    for i in 1:length(scene_list)
        scene_visual = render(scene_list[i],ROADWAY,
        [IDOverlay(colorant"white",12)],
#         cam=FitToContentCamera(0.),
        cam = CarFollowCamera(1)
        )
        push!(frames,scene_visual)
    end
    print("Making video filename: $(filename)\n")
    write(filename,frames)
    return nothing
end

"""
    function scenelist2video
- Make video from two different scene lists overlaying `scene_list_2` in blue on top of `scene_list_1` in pink

# Examples
```julia
multiple_scenelist2video(true_scene_list,imit_scene_list,filename="media/mobil/true_vs_imitation.mp4")
```
"""
function multiple_scenelist2video(scene_list_1,scene_list_2;
    filename = "media/mobil/multiple_scene_to_video.mp4")
    frames = Frames(MIME("image/png"),fps = 10)
    @assert length(scene_list_1) == length(scene_list_2)
    # Loop over list of scenes and convert to video
    for i in 1:length(scene_list_1)
        other_overlay = my_overlay(scene_list_2[i],colorant"blue")
        scene_visual = render(scene_list_1[i],ROADWAY,
        [IDOverlay(colorant"white",12),other_overlay,
                TextOverlay(text=["Imitation in blue"],font_size=14)],
#         cam=FitToContentCamera(0.),
        cam = CarFollowCamera(1)
        )
        push!(frames,scene_visual)
    end
    print("Making video filename: $(filename)\n")
    write(filename,frames)
    return nothing
end

"""
- Make a video only taking every 5 timesteps into account from `scene_list`

# Examples
```julia
start_scene = deepcopy(scene_list[10])
scene_list_quantization = get_hallucination_scenes(start_scene; nsteps=90,models=models)
```
"""
function scenelist2video_quantized(scene_list;
    filename = "media/mobil/scene_to_video.mp4")
    frames = Frames(MIME("image/png"),fps = 5)
    
    # Loop over list of scenes and convert to video
    for i in 1:length(scene_list)
	if i%5 == 0
		scene_visual = render(scene_list[i],ROADWAY,
		[IDOverlay(colorant"white",12),TextOverlay(text=["frame=$(i)"],font_size=12)],
	#         cam=FitToContentCamera(0.),
		cam = CarFollowCamera(1)
		)
		push!(frames,scene_visual)
	end
    end
    print("Making video filename: $(filename)\n")
    write(filename,frames)
    return nothing
end

scenelist2video_quantized

In [6]:
# functions [plots]:ytrace,lanes,overlay ytraces
"""
    function scenelist2ytrace(scene_list;car_id=-1)
- Make a y position trace from a list of scenes

# Examples
```julia
scenelist2ytrace(scene_list,car_id=1)
```
"""
function scenelist2ytrace(scene_list;car_id=-1)
    if car_id == -1 print("Please provide a valid car_id\n") end

    p = PGFPlots.Plots.Scatter(collect(1:length(scene_list)),
        [scene[findfirst(car_id,scene)].state.posG.y for scene in scene_list],legendentry="y trace")
    return p
end

"""
    function overlay_jld_scenelists(num_exps)
- To assess impact of headway sensor noise
- Capture scene_lists from stored .jld files of `num_exps` experiments

# Examples
``` julia
# Suppose we have 1.jld, 2.jld,...,8.jld
overlay_jld_scenelists(8)
```
"""
function overlay_jld_scenelists(num_exps)
    horizon = 100
    list_of_plots = PGFPlots.Plot[]
    for i in 1:num_exps
        scene_list = JLD.load("media/mobil/$(i).jld","scene_list")
        p = PGFPlots.Plots.Scatter(collect(1:horizon),
            [scene[1].state.posG.y for scene in scene_list[1:horizon]],
            legendentry = "scenario $(i)")
        push!(list_of_plots,p)
    end
    pa = PGFPlots.Axis(list_of_plots,xlabel="timestep",ylabel="y pos",
        legendPos="outer north east")
    display(pa)
    PGFPlots.save("media/mobil/sensor_noise_impact.pdf",pa)
    return nothing
end

"""
    function plot_lanes(scene_list_1)
- Goal is to plot lanes on a y trace of vehicles using `scene_list_1`

# Examples
```julia
plot_lanes(scene_list)
```
"""
function plot_lanes(scene_list_1)
    laneboundary0 = PGFPlots.Plots.Linear(x[:,1],fill(-1.5,length(scene_list_1)),
        style="black,ultra thick")
    laneboundary1 = PGFPlots.Plots.Linear(x[:,1],fill(1.5,length(scene_list_1)),style="black,ultra thick")
    laneboundary2 = PGFPlots.Plots.Linear(x[:,1],fill(4.5,length(scene_list_1)),style="black,ultra thick")
    return [laneboundary0,laneboundary1,laneboundary2]
end

plot_lanes

In [7]:
# function: hallucinate scene list
"""
    function get_hallucination_scenes
- Hallucinate starting from `start_step` for `nsteps` using `models` and return a list of scenes
- Used by `plot_carwise_pos_vel` to assess position and velocity traces against ground truth

# Returns
- `halluc_scenes_list`: List containing the scenes starting with the ground truth scene at `start_step`

# Examples
```julia
true_scene_list = get_hallucination_scenes(scene_halluc,duration=10,models=models);
```
"""
function get_hallucination_scenes(start_scene;models,start_step=1,duration=10,
        id_list=[],verbosity = false,timestep=TIMESTEP)
        # Setting up
    scene_halluc = start_scene
    halluc_scenes_list = []
    push!(halluc_scenes_list,deepcopy(start_scene))
#     scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
#     push!(halluc_scenes_list,deepcopy(scene_halluc))
    
    nsteps = duration/timestep
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
        
#         if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end
        
        actions = Array{Any}(undef,length(scene_halluc))

            # Propagation of scene forward
        get_actions!(actions,scene_halluc,ROADWAY,models)

        tick!(scene_halluc,ROADWAY,actions,timestep)
        
        push!(halluc_scenes_list,deepcopy(scene_halluc))
    end 
    return halluc_scenes_list
end

get_hallucination_scenes

In [8]:
# function: Generate uniform sampling to start the initial particle matrix
"""
    function initial_pmat(;limits,num_particles,seed)
- Generate initial particle matrix with `num_particles` particles with every col being a diff particle
- Range of values that parameters can take is specified in `limits`. Should be num_params rows x 2 cols

# Examples
```julia
limits = [10. 40.;0.1 10.;0.5 5.;1. 10.;0. 1.;-1. 1.;-20. 20.]
initial_pmat(limits=limits,num_particles=10,seed=4)
```
"""
function initial_pmat(;limits,num_particles,seed)
    Random.seed!(seed)
    num_params = size(limits,1)
    p_mat = fill(0.,num_params,num_particles)

    for i in 1:num_params
        p_mat[i,:] = rand(Uniform(limits[i,1],limits[i,2]),1,num_particles)
    end
    return p_mat
end

initial_pmat

In [9]:
# function: hallucinate_a_step
"""
    function hallucinate_a_step(scene_input,particle;car_id=-1)
- Hallucinate one step starting from `scene_input` using parameters given by `particle`

# Examples
```julia
limits = [10. 40.;0.1 10.;0.5 5.;1. 10.;0. 1.;-1. 1.;0. 20.]
init_pmat = initial_pmat(limits=limits,num_particles=10,seed=4)
hallucinate_a_step(SCENE,init_pmat[:,9],car_id=1)
```
"""
function hallucinate_a_step(scene_input,particle;car_id=-1)
    if car_id==-1 @show "hallucinate_a_step says: Please give a valid car_id" end

    scene = deepcopy(scene_input)
    models = Dict{Int64,DriverModel}()

    for veh in scene
        if veh.id == car_id
            models[veh.id] = Tim2DDriver(TIMESTEP,
                                    mlane=MOBIL(TIMESTEP,politeness=particle[POLITENESS],
                                                advantage_threshold=particle[ADV_TH],
                                                mlon=uncertain_IDM(sigma_sensor=particle[SENSOR_SIGMA])
                                    ),
                                    mlon = IntelligentDriverModel(v_des=particle[V_DES],σ=particle[SIGMA_IDM],
                                            T=particle[T_HEADWAY],s_min=particle[S_MIN]
                                    )
                            )
        else
            models[veh.id] = IntelligentDriverModel(v_des=50.)
        end
    end

    actions = Array{Any}(undef,length(scene))
    get_actions!(actions,scene,ROADWAY,models)
    tick!(scene,ROADWAY,actions,TIMESTEP)

    halluc_state = scene.entities[findfirst(car_id,scene)].state
    halluc_pos = halluc_state.posF.s
    halluc_lane = get_lane_id(scene,car_id)

    return halluc_pos,halluc_lane
end

hallucinate_a_step

In [10]:
# function: weight and resample
"""
    function weight_and_resample(start_scene,true_nextpos,true_nextlane,p_mat;car_id=-1)
- Hallucination from `start_scene` 
- Compare against ground truth at `true_nextpos`, `true_nextlane`
- Assign weights to particles
- Perform resampling and return a new matrix of particles and associated weight vector

# Examples
```julia
# Test for one step on initial particle matrix
limits = [10. 40.;0.1 10.;0.5 5.;1. 10.;0. 1.;-1. 1.;0. 20.]
init_pmat = initial_pmat(limits=limits,num_particles=10,seed=4)
id = 1
true_next_scene = deepcopy(true_scene_list[2])
true_nextpos = get_frenet_s(true_next_scene;car_id=id)
true_nextlane = get_lane_id(true_next_scene,id)
weight_and_resample(SCENE,true_nextpos,true_nextlane,init_pmat,car_id=id)
```
"""
function weight_and_resample(start_scene,true_nextpos,true_nextlane,p_mat;car_id=-1,verbosity=false)
    if car_id==-1 @show "compute_particle_likelihood says: Please give valid car_id" end
    num_p = size(p_mat,2)
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p
        if verbosity print("w_and_resample says: particle number = $i \n") end
        particle = p_mat[:,i]
        
        std_dev_acc = p_mat[SIGMA_IDM]
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        std_dev_pos = TIMESTEP*TIMESTEP*std_dev_acc
        hpos,hlane = hallucinate_a_step(start_scene,particle,car_id=car_id)
        
        start_lane = get_lane_id(start_scene,car_id)
        lane_has_changed = false

        if start_lane != true_nextlane
            lane_has_changed = true
        end

        p_lanechange = get_lane_change_prob(start_scene,particle,car_id=car_id)

        prob_lane = 0.5 # Initialize to random
        if lane_has_changed
            prob_lane = p_lanechange
        else
            prob_lane = 1-p_lanechange
        end
        
        prob_pos = pdf(Normal(hpos,std_dev_pos),true_nextpos)
        if verbosity
            print("weight and resample says: true_nextpos = $(true_nextpos) and hpos=$(hpos) and hlane=$(hlane)\n")
            print("weight and resample says: prob_pos = $(prob_pos) and prob_lane=$(prob_lane)\n")
        end
        lkhd_vec[i] = prob_lane*prob_pos
    end
    p_weight_vec = StatsBase.weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
    
    idx = sample(1:num_p,p_weight_vec,num_p)
    if verbosity print("weight and resample says: ids are $(idx)\n") end
    new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
    
    return new_p_mat, p_weight_vec
end

weight_and_resample

In [11]:
# function: multistep_update
"""
- Run filtering over a trajectory by starting from a true scene
- Repeatedly calls `weight_and_resample` on a demonstration trajectory

# Caveats
- Hard coded usage of `true_scene_list` which is basically ground truth generated by `get_hallucination_scenes`
- Hard coded limits on initial particle distribution generation

# Returns
- `final_p_mat`: Matrix with particles in separate columns
- `iterwise_p_mat`: A list with the associated particle matrix at each iteration

# Examples
```julia
final_p_mat,iterwise_p_mat = multistep_update(car_id=1,start_frame=2,last_frame=99)
```
"""
function multistep_update(;car_id,start_frame,last_frame,num_p=500,seed=1,verbosity=false)
    if verbosity print("car id = $(car_id)\n") end
        
        # Careful selectin of particles around initial velocity of data
        # Commented out for now as we are not yet at the stage of using TRAJ
    #startscene = get_scene(start_frame,TRAJ)
    #startpos,startvel = get_veh_info(startscene,car_id=car_id)
    #init_p_mat = sample_init_particles(num_p,v=startvel)

    limits = [10. 40.;0.1 10.;0.5 5.;1. 10.;0. 1.;-1. 1.;0. 20.]
    p_mat = initial_pmat(limits=limits,num_particles=num_p,seed=seed)
    iterwise_p_set = [] # Stores particle set at every iteration
    push!(iterwise_p_set,p_mat)
    
    for framenum in start_frame:last_frame
            # Get the truth from TRAJ
        #scene = get_scene(framenum+1,TRAJ)
        #trupos,truvel = get_veh_info(scene,car_id=car_id,traj=traj)
        
            # Get the scene to start hallucinating from
        #scene = get_scene(framenum,TRAJ)
        scene = deepcopy(true_scene_list[framenum])
        true_next_scene = deepcopy(true_scene_list[framenum+1])
        true_nextpos = get_frenet_s(true_next_scene;car_id=car_id)
        true_nextlane = get_lane_id(true_next_scene,car_id)
        # print("multistep update says: true_nextpos = $(true_nextpos), true_nextlane=$(true_nextlane)\n")
        p_mat_new, weight_vec = weight_and_resample(scene,true_nextpos,true_nextlane,p_mat,car_id=car_id)

            # Addition of dithering noise
        #params = [:v_des,:σ]
        #p_mat = addnoise(p_mat_new, weight_vec)
        
        p_mat = p_mat_new # No noise addition for now
        
            # Storage into list that contains particle matrix at every step
        push!(iterwise_p_set,p_mat)
    end

        # Find the mean particle after filtering is over
        # Defined in `admin.jl`
    mean_particle = mean(p_mat,dims=2)
    
    return mean_particle,iterwise_p_set
end

multistep_update

In [12]:
# function: gen_imitation_traj
"""
    function gen_imitation_traj(p_mat)
- Generate an imitation trajectory using the mean particle obtained from `p_mat`
- `p_mat` is the resulting particle matrix after running `multistep_update`
i.e. obtained after running particle filtering over a trajectory of length `duration`

# Examples
```julia
start_scene = deepcopy(SCENE)
imit_scene_list = gen_imitation_traj(final_p_mat,start_scene,car_id=1,timestep=0.02)
```
"""
function gen_imitation_traj(p_mat,start_scene;start_step=1,duration=10,id_list=[],car_id=-1,timestep=TIMESTEP)
    particle = mean(p_mat,dims=2)
    models = Dict{Int64,DriverModel}()

    for veh in scene
        if veh.id == car_id
            models[veh.id] = Tim2DDriver(timestep,
                                    mlane=MOBIL(timestep,politeness=particle[POLITENESS],
                                                advantage_threshold=particle[ADV_TH],
                                                mlon=uncertain_IDM(sigma_sensor=particle[SENSOR_SIGMA])
                                    ),
                                    mlon = IntelligentDriverModel(v_des=particle[V_DES],σ=particle[SIGMA_IDM],
                                            T=particle[T_HEADWAY],s_min=particle[S_MIN]
                                    )
                            )
        else
            models[veh.id] = IntelligentDriverModel(v_des=15.)
        end
    end
    models[3] = IntelligentDriverModel()
    
            # Setting up
    scene_halluc = start_scene
    halluc_scenes_list = []
    push!(halluc_scenes_list,deepcopy(start_scene))
#     scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
#     push!(halluc_scenes_list,deepcopy(scene_halluc))
    nsteps = Int64(floor(duration/timestep))
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
        
#         if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end
        
        actions = Array{Any}(undef,length(scene_halluc))

            # Propagation of scene forward
        get_actions!(actions,scene_halluc,ROADWAY,models)

        tick!(scene_halluc,ROADWAY,actions,timestep)
        
        push!(halluc_scenes_list,deepcopy(scene_halluc))
    end 
    return halluc_scenes_list
end

gen_imitation_traj

In [13]:
# function: compute rmse of generated trajectory vs true trajectory
"""
    function compute_rmse(true_scene_list,halluc_scene_list;id_list)
- Compute rmse position and velocity between `halluc_scene_list` and `true_scene_list`
- `true_scene_list` is the ground truth demonstration trajectory
- `imit_scene_list` is generated by running vehicles using parameters that we want for the driver model

# Returns
- `rmse_pos::Dict{Int64,Vector{Float64}}`: Key is vehid. Value is array. Each elem is timewise rmse pos value for that veh_id
- `rmse_vel::Dict{Int64,Vector{Float64}}`: Key is vehid. Value is array. Each elem is timewise rmse vel value for that veh_id

# Examples
```julia
rmse_pos_dict,rmse_vel_dict = compute_rmse(true_scene_list,imit_scene_list,id_list=[1,2,3])
```
"""
function compute_rmse(true_scene_list,imit_scene_list;id_list=[])
    @assert length(true_scene_list) == length(imit_scene_list)
    rmse_pos = Dict{Int64,Vector{Float64}}()
    rmse_vel = Dict{Int64,Vector{Float64}}()
    
    for veh_id in id_list
        rmse_pos[veh_id] = []
        rmse_vel[veh_id] = []
    end
    
    for i in 1:length(true_scene_list)
        scene_halluc = imit_scene_list[i]
        demo_scene_target = true_scene_list[i]
        
        for veh_id in id_list
            demo_veh = demo_scene_target[findfirst(veh_id,demo_scene_target)]
            ego_veh = scene_halluc[findfirst(veh_id,scene_halluc)]

            push!(rmse_pos[veh_id],norm(demo_veh.state.posG[1:2]-ego_veh.state.posG[1:2]))
            push!(rmse_vel[veh_id],norm(demo_veh.state.v - ego_veh.state.v))
        end
    end
    return rmse_pos,rmse_vel
end

compute_rmse

In [14]:
# function: Combine carwise rmse into one metric by averaging over cars
"""
    function rmse_dict2mean(rmse_dict)
- Take dict of carwise rmse value and return an array with mean of rmse taken over cars

# Examples
```julia

```
"""
function rmse_dict2mean(rmse_dict)
    num_veh = length(collect(keys(rmse_dict))) # Find length of the vector of keys
    num_iter = length(rmse_dict[collect(keys(rmse_dict))[1]]) # Find length of value contained in 1st key
    rmse_array = fill(0.,num_iter,num_veh)

    i = 0
    for (k,v) in rmse_dict
        i = i+1
        rmse_vals = reshape(v,length(v),1)
        rmse_array[:,i] = rmse_vals
    end

    carmean_rmse = mean(rmse_array,dims=2)
    return carmean_rmse
end

rmse_dict2mean

In [15]:
# function: plot particle progres optionally with errorbars
"""
    function plot_p_prog(iterwise_p_mat;make_error_bars = false)
- Makes plot of mean particles from list of particle matrices over iterations of filtering
- Makes error bars depending on the `make_error_bars` kwarg

# Examples
```julia
final_p_mat,iterwise_p_mat = multistep_update(car_id=1,start_frame=2,last_frame=99);
plot_p_prog(iterwise_p_mat,make_error_bars=true,display_plot=true)
```
"""
function plot_p_prog(iterwise_p_mat;make_error_bars = false,display_plot=false)
    num_steps = length(iterwise_p_mat)
    num_params = size(iterwise_p_mat[1],1)
    mean_p = fill(0.,num_params,num_steps)
    max_p = fill(0.,num_params,num_steps)
    min_p = fill(0.,num_params,num_steps)
    for i in 1:num_steps
        mat = iterwise_p_mat[i]
        mean_p[:,i] = mean(mat,dims=2) # Take the mean of the sample of particles
        max_p[:,i] = maximum(mat,dims=2)
        min_p[:,i] = minimum(mat,dims=2)
    end
    plot_list = PGFPlots.Plots.Plot[] # Initialize empty array of type PGFPlots.Plots.Plot
    for i in 1:num_params
        if make_error_bars
            push!(plot_list,PGFPlots.Plots.Linear(collect(1:num_steps),mean_p[i,:],
                    errorBars = ErrorBars(yplus=max_p[i,:], yminus=min_p[i,:]),
                    legendentry=param_names[i],onlyMarks=true))
        else
            push!(plot_list,PGFPlots.Plots.Linear(collect(1:num_steps),mean_p[i,:],
                    legendentry=param_names[i],onlyMarks=true))
        end
    end
    pa = PGFPlots.Axis(plot_list,xlabel="timestep",ylabel="param value",
        title="param wise filtering progress",legendPos="outer north east")
    if display_plot display(pa) end
    return pa
end

plot_p_prog

In [16]:
# function: extract xy trace from list of scenes
"""
    function scene_list2pos_trace(scene_list;ncars,filename="")
- Written with the goal of giving pos trace to Alex
- Writes the xy positions of first `ncars` in the format x1,y1,x2,y2,...,xn,yn cols of data
- Optinonally writes the data to a csv file with `filename`

# Examples
```julia
scene_list2pos_trace(alex_true_scene_list,ncars=3,filename="media/mobil/ground_truth.csv")
```
"""
function scene_list2pos_trace(scene_list;ncars,filename="")
data = fill(0.,length(scene_list),2*ncars) # carwise x, y
    for (i,scene) in enumerate(scene_list)
        for j in 1:ncars
            data[i,j*2-1] = scene[j].state.posG.x
            data[i,j*2] = scene[j].state.posG.y
        end
    end
    
    if filename != "" writedlm(filename,data,",") end
    return data
end

scene_list2pos_trace

In [ ]:
# function: pgfplots 2 gif using Reel
"""
    function pgfplots2gif(plots;filename="output.gif")
- Make a video using an array of plots. Uses the Reel library.

# Caveat
- Only works with .gif output type. If use .mp4, errors saying process exited ffmpeg
"""
function pgfplots2gif(plots;filename="output.gif")
    @assert typeof(filename) == String
    frames = Frames(MIME("image/svg+xml"), fps=10)
    
    for plt in plots
        push!(frames, plt)
    end
    write(filename, frames)
    return nothing
end # End of the reel gif writing function

In [ ]:
# function: make pairwise particle variation videos, what pairs to make is the question
"""
- We have a list with corresponding particle matrix at every iteration of filtering
- We shall scatter two columns of said matrix
- We shall make a video with said scatter at every timestep

# Examples
```julia
seed = 2; num_p = 500
Random.seed!(seed)
final_p_mat,iterwise_p_mat = multistep_update(car_id=1,start_frame=2,last_frame=99,num_p=num_p);
plot_pairwise_particles(iterwise_p_mat)
```
"""
function plot_pairwise_particles(iterwise_p_mat)
    num_params = size(iterwise_p_mat[1],1) # Number of rows tells us the number of parameters
    pairs = collect(combinations(collect(1:num_params),2))
    num_pairs = length(pairs)
    groupplots = PGFPlots.GroupPlot[]
    
    for i in 1:length(iterwise_p_mat)-70
        print("plot_pairwise_particles says iteration number = $i\n")
        g = PGFPlots.GroupPlot(5,4)
        p_mat = iterwise_p_mat[i]
        
        for j in 1:num_pairs-1
            kk = pairs[j]
            param_1 = kk[1];param_2 = kk[2]
            name_1 = param_names[param_1];name_2=param_names[param_2]
            p = PGFPlots.Axis([PGFPlots.Plots.Scatter(p_mat[param_1,:],p_mat[param_2,:])],
                    xlabel=name_1,ylabel=name_2)
            push!(g,p)
        end
        push!(groupplots,g)
    end
    pgfplots2gif(groupplots,filename="media/mobil/pairwise_vids/seed2.gif")
end

#### global definitions for initialization; ground truth data generation

In [37]:
# GLOBAL DEFINITIONS: TIMESTEP, ROADWAY, PARTICLE INDICES
const TIMESTEP = 0.1;
const V_DES = 1; const SIGMA_IDM = 2; const T_HEADWAY = 3; const S_MIN=4; 
const POLITENESS = 5;const ADV_TH = 6;const SENSOR_SIGMA = 7;
const DEFAULT_PARAMS = [29.,NaN,1.5,5.,0.35,0.1,NaN] # The parameters set to default values

pos_vel_array_1 = [(200.,30.),(250.,0.)]
pos_vel_array_2 = [(250.,10.)] #(280.,10.)
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2]
scene,roadway = init_place_cars(lane_place_array)
const SCENE = deepcopy(scene)
const ROADWAY = roadway;

# Stores the name of the parameter corresponding to number. Helpful in making legend when we plot param prog
param_names = Dict(1=>"Desired velocity",2=>"Acceleration output noise",3=>"Min time headway",
    4=>"Min separation",5=>"Politeness",6=>"Advantage threshold",7=>"Headway sensor noise");

# Set the Reel extension to work with pgfplots, https://github.com/JuliaTeX/PGFPlots.jl/issues/40
Reel.extension(m::MIME"image/svg+xml") = "svg"

In [18]:
# GROUND TRUTH DATA: Default model params 100 timesteps, returns true_scene_list
start_step=1
nsteps=100
scene_halluc = deepcopy(SCENE)
models = Dict{Int64,DriverModel}()
for veh in scene models[veh.id] = IntelligentDriverModel() end
models[1] = Tim2DDriver(TIMESTEP,
                        mlane=MOBIL(TIMESTEP),
                        mlon=IntelligentDriverModel(),
            )
models[2] = IntelligentDriverModel(v_des=15.)
models[3] = IntelligentDriverModel()

true_scene_list = get_hallucination_scenes(scene_halluc,duration=10,models=models);
# scenelist2video(true_scene_list,filename="media/mobil/ground_truth.mp4")

#### wip, development space

# Scripts

In [ ]:
# Script: Filter, imitate, rmse, overlay video
seed = 2; num_p = 500
Random.seed!(seed)
final_p_mat,iterwise_p_mat = multistep_update(car_id=1,start_frame=2,last_frame=99,num_p=num_p);
start_scene = deepcopy(SCENE)
imit_scene_list = gen_imitation_traj(final_p_mat,start_scene,car_id=1)
rmse_pos_dict,rmse_vel_dict = compute_rmse(true_scene_list,imit_scene_list,id_list=[1,2,3])
rmse_pos = rmse_dict2mean(rmse_pos_dict);rmse_vel = rmse_dict2mean(rmse_vel_dict)
rmse_pos = reshape(rmse_pos,length(rmse_pos),);rmse_vel = reshape(rmse_vel,length(rmse_vel),)
p_pos = PGFPlots.Plots.Linear(collect(1:length(rmse_pos)),rmse_pos,legendentry="rmse pos")
p_vel = PGFPlots.Plots.Linear(collect(1:length(rmse_vel)),rmse_vel,legendentry="rmse vel")
rmse_axis = PGFPlots.Axis([p_pos,p_vel],xlabel="timestep",ylabel="rmse",title="rmse pos and vel")
display(rmse_axis)
multiple_scenelist2video(true_scene_list,imit_scene_list,
    filename="media/mobil/true_vs_imit_faster_nump_$(num_p)_seed_$(seed).mp4")

In [ ]:
# Script: Generate driving trajectories with different random seeds using mean particle
seeds = collect(1:10)
car_id = 1
y_trace_plot = PGFPlots.Plots.Plot[]
lanes_plot = PGFPlots.Plots.Plot[]
for seed in seeds
    start_scene = deepcopy(SCENE)
    Random.seed!(seed)
    scene_list = gen_imitation_traj(final_p_mat,start_scene,start_step=1,nsteps=100,car_id=1)
    push!(y_trace_plot,PGFPlots.Plots.Scatter(collect(1:length(scene_list)),
                [scene[car_id].state.posG.y for scene in scene_list],legendentry="seed = $seed"))
    #multiple_scenelist2video(true_scene_list,scene_list,
    #    filename = "media/mobil/true_vs_imitation_seed_$(seed).mp4")
    
end
push!(y_trace_plot,PGFPlots.Plots.Scatter(collect(1:length(true_scene_list)),
        [true_scene[car_id].state.posG.y for true_scene in true_scene_list],legendentry="ground truth"))
PGFPlots.Axis(y_trace_plot,xlabel="timestep",ylabel="y pos trace",
    title="Mean filtered particle y pos trace",legendPos="outer north east")

In [ ]:
# Script: Different timestep values for Alex carla
start_scene = deepcopy(SCENE)
scene_list = gen_imitation_traj(final_p_mat,start_scene,car_id=1,timestep=0.02)
start_scene = deepcopy(SCENE)
scene_list_2 = gen_imitation_traj(final_p_mat,start_scene,car_id=1,timestep=0.1)
p1 = PGFPlots.Plots.Scatter([scene[1].state.posG.x for scene in scene_list],
    [scene[1].state.posG.y for scene in scene_list],legendentry="timestep 0.02")
p2 = PGFPlots.Plots.Scatter([scene[1].state.posG.x for scene in scene_list_2],
    [scene[1].state.posG.y for scene in scene_list_2],legendentry = "timestep 0.1")
PGFPlots.Axis([p1,p2],xlabel="x",ylabel="y")

In [ ]:
# Script: Generate ground truth data for Alex
start_step=1
scene_halluc = deepcopy(SCENE)
models = Dict{Int64,DriverModel}()
for veh in scene models[veh.id] = IntelligentDriverModel() end
models[1] = Tim2DDriver(TIMESTEP,
                        mlane=MOBIL(TIMESTEP),
                        mlon=IntelligentDriverModel(),
            )
models[2] = IntelligentDriverModel(v_des=15.)
models[3] = IntelligentDriverModel()

alex_true_scene_list = get_hallucination_scenes(scene_halluc,duration=10,models=models,timestep=0.02);
p1 = PGFPlots.Plots.Scatter([scene[1].state.posG.x for scene in scene_list],
    [scene[1].state.posG.y for scene in scene_list],legendentry="imitation")
p2 = PGFPlots.Plots.Scatter([scene[1].state.posG.x for scene in alex_true_scene_list],
    [scene[1].state.posG.y for scene in alex_true_scene_list],legendentry = "ground truth")
pa = PGFPlots.Axis([p1,p2],xlabel="x",ylabel="y")
PGFPlots.save("media/mobil/trace_for_carla.svg",pa)
multiple_scenelist2video(alex_true_scene_list,scene_list,filename="media/mobil/2d_for_carla.mp4")

In [ ]:
# Script: Generate lane change prob and y pos trace and overlay the two
particle = [29.0,0.5,1.2,1.5,0.,0.1,20.]
start_step = 1; nsteps = 100
scene_halluc = deepcopy(scene)
models = Dict{Int64,DriverModel}()
for veh in scene
    models[veh.id] = uncertain_IDM(sigma_sensor=20.)
end
models[1] = Tim2DDriver(TIMESTEP,mlane=MOBIL(TIMESTEP,politeness=particle[POLITENESS],
        mlon=uncertain_IDM(sigma_sensor=particle[SENSOR_SIGMA])),
    mlon = uncertain_IDM(sigma_sensor=particle[SENSOR_SIGMA]),)
models[2] = uncertain_IDM(v_des=15.,sigma_sensor=particle[SENSOR_SIGMA])

halluc_scenes_list = []
lc_probs = fill(0.,nsteps,)
id = 1
for (i,t) in enumerate(start_step:start_step+nsteps-1)
    print("test_laneprob.jl says: t=$t\n")
    lp = get_lane_change_prob(scene_halluc,particle,car_id=id)
    #print("test_laneprob.jl says lane change prob = $(lp)\n")
    actions = Array{Any}(undef,length(scene_halluc))
    get_actions!(actions,scene_halluc,ROADWAY,models)
    tick!(scene_halluc,ROADWAY,actions,TIMESTEP)
    push!(halluc_scenes_list,deepcopy(scene_halluc))
    lc_probs[i] = lp
end

plot_ytrace = scenelist2ytrace(halluc_scenes_list,car_id=id)
plot_probs = PGFPlots.Plots.Scatter(collect(1:nsteps),lc_probs,legendentry="lane change probs")
pa = PGFPlots.Axis([plot_ytrace,plot_probs],xlabel="timestep",ylabel="ytrace and lc probs",
    legendPos="outer north east",title="Lane change prob and y pos trace for car id=$(id)")
display(pa)
#PGFPlots.save("media/mobil/lc_ytrace.pdf",)

In [ ]:
# Script: Compare two politeness values in one video by overlaying
pos_vel_array_1 = [(200.,30.),(250.,0.)]
pos_vel_array_2 = [(250.,10.)]
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2]
scene,roadway = init_place_cars(lane_place_array)
start_scene = deepcopy(scene)
models_1 = Dict{Int64,DriverModel}()
models_2 = Dict{Int64,DriverModel}()
for veh in scene
    models_1[veh.id] = IntelligentDriverModel()
    models_2[veh.id] = IntelligentDriverModel()
end
politeness = 0.
models_1[1] = Tim2DDriver(timestep_ngsim,mlane=MOBIL(timestep_ngsim,politeness=politeness))
models_1[2] = IntelligentDriverModel(v_des=15.)

scene_list_1 = get_hallucination_scenes(scene,nsteps=100,models=models_1,roadway=roadway)

politeness = 1.
models_2[1] = Tim2DDriver(timestep_ngsim,mlane=MOBIL(timestep_ngsim,politeness=politeness))
models_2[2] = IntelligentDriverModel(v_des=15.)

scene_list_2 = get_hallucination_scenes(start_scene,nsteps=100,models=models_2,roadway=roadway)
multiple_scenelist2video(scene_list_1,scene_list_2,roadway=roadway,
    filename="media/mobil/compare_politeness.mp4")

In [ ]:
# Script: Compare two advantage_threshold values in one video by overlaying
pos_vel_array_1 = [(200.,30.),(250.,0.)]
pos_vel_array_2 = [(250.,10.)]
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2]
scene,roadway = init_place_cars(lane_place_array)
start_scene = deepcopy(scene)
models_1 = Dict{Int64,DriverModel}()
models_2 = Dict{Int64,DriverModel}()
for veh in scene
    models_1[veh.id] = IntelligentDriverModel()
    models_2[veh.id] = IntelligentDriverModel()
end
politeness = 0.
a_th = 0.
models_1[1] = Tim2DDriver(timestep_ngsim,
    mlane=MOBIL(timestep_ngsim,politeness=politeness,advantage_threshold=a_th))
models_1[2] = IntelligentDriverModel(v_des=15.)

scene_list_1 = get_hallucination_scenes(scene,nsteps=100,models=models_1,roadway=roadway)

politeness = 0.
a_th = 1.
models_2[1] = Tim2DDriver(timestep_ngsim,
    mlane=MOBIL(timestep_ngsim,politeness=politeness,advantage_threshold=a_th))
models_2[2] = IntelligentDriverModel(v_des=15.)

scene_list_2 = get_hallucination_scenes(start_scene,nsteps=100,models=models_2,roadway=roadway)
multiple_scenelist2video(scene_list_1,scene_list_2,roadway=roadway,
    filename="media/mobil/compare_adv_blueHighThreshold_p_0.mp4")

In [ ]:
# Script: Influence of stochasticity on lateral driving
seed=3;
Random.seed!(seed) # To control the stochasticity in ProportionalLaneTracker

pos_vel_array_1 = [(200.,30.),(250.,0.)]
pos_vel_array_2 = [(250.,10.)]
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2]
scene,roadway = init_place_cars(lane_place_array)
start_scene = deepcopy(scene)
models_1 = Dict{Int64,DriverModel}()
models_2 = Dict{Int64,DriverModel}()
for veh in scene
    models_1[veh.id] = IntelligentDriverModel()
    models_2[veh.id] = IntelligentDriverModel()
end
models_1[1] = Tim2DDriver(timestep_ngsim,
    mlane=MOBIL(timestep_ngsim),
    mlat=ProportionalLaneTracker()
)
models_1[2] = IntelligentDriverModel(v_des=15.)

scene_list_1 = get_hallucination_scenes(scene,nsteps=100,models=models_1,roadway=roadway)

sigma_lat = 10.
models_2[1] = Tim2DDriver(timestep_ngsim,
    mlane=MOBIL(timestep_ngsim),
    mlat=ProportionalLaneTracker(σ=sigma_lat)
)
models_2[2] = IntelligentDriverModel(v_des=15.)

scene_list_2 = get_hallucination_scenes(start_scene,nsteps=100,models=models_2,roadway=roadway)
multiple_scenelist2video(scene_list_1,scene_list_2,roadway=roadway,
    filename="media/mobil/lateral_blueIsStochastic_$(seed)_seed_sigmaLat_$(sigma_lat).mp4")

# Playground

#### bivariate normal to experiment with the noise in proportional lane tracker

In [ ]:
d = MvNormal([0.,0.],[1. 0.;0. 1.])

In [ ]:
pdf(d,[0.5,-0.5])

In [ ]:
# Lets think of a hybrid discrete and continuous distribution
b = Bernoulli(0.3)
rand(b,50)
pdf(b,0)
d = Normal(0,1)
pdf(b,0)*pdf(d,0.4)